### Import Frameworks + Libraries

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import os


### Pointer Logic

- Create pointer variables to save .html files to a particular directory

In [ ]:
SCORE_DIR = "data/scores"
box_scores = os.listdir(SCORE_DIR)
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]

### Parse HTML Function

- Takes in a particular HTML file for a specific game, and removes excess table data that is not required in the df (s.decompose())

In [ ]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)

    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]

    return soup
    

### Parse Line Score

- Takes in a BeautifulSoup object to parse the total points/teams playing

In [ ]:
def read_line_score(soup):
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols

    line_score = line_score[["team", "total"]]
    return line_score

### Read Stats

- Reads in the boxscore HTML table and converts it to a dataframe using Pandas

In [ ]:
def read_stats(soup, team, stat) -> pd.DataFrame:
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")

    return df

### Read Season Info

- Returns the particular season info of the game we are currently parsing

In [ ]:
def read_season_info(soup) -> pd.DataFrame:
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all("a")]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [ ]:
games = []

for box_score in box_scores:
    soup = parse_html(box_score)
    line_score = read_line_score(soup)

    teams = list(line_score["team"])
    summaries = []
    team_mapping = {}

    for i, team in enumerate(teams):
        team_mapping[f"team{i}"] = team
        basic = read_stats(soup, team, "basic")
        advanced = read_stats(soup, team, "advanced")

        # only take the final row in the DataFrame and concatenate basic and advanced stats 
        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])

        totals.index = totals.index.str.lower() + f"_team{i}"
            
        totals[f"team{i}"] = team
        summaries.append(totals)

    game = pd.concat(summaries, axis=0)
    game["winner"] = team_mapping["team0"] if game["pts_team0"] > game["pts_team1"] else team_mapping["team1"]
    game = game.to_frame().T

    game["season"] = read_season_info(soup)
    game["date"] = os.path.basename(box_score)[:8]
    game["date"] = pd.to_datetime(game["date"], format="%Y%m%d")
    games.append(game)

    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(box_scores)}")

In [ ]:
games = [df.reset_index(drop=True) for df in games]
games_df = pd.concat(games, axis=0)
games_df.reset_index(drop=True, inplace=True)

games_df

In [ ]:
games_df = games_df.sort_values(by="date")
games_df = games_df.reset_index(drop=True)

In [ ]:
games_df

## Encoding Data

- label encoding --> went with this one for now, but am not sure if this is good since it implies some type of ordering which is not true
- one-hot encoding --> this one will add 30 columns to the df, is this what we want?

In [ ]:
# need to confirm previous years do not have different team encodings

team_encoding = { 
    # ATLANTIC
    "TOR": 1,
    "BOS": 2,
    "NYK": 3, 
    "BRK": 4,
    "PHI": 5,

    # CENTRAL
    "CLE": 6,
    "IND": 7,
    "DET": 8,
    "CHI": 9,
    "MIL": 10,

    # SOUTHEAST
    "MIA": 11,
    "ATL": 12,
    "CHO": 13,
    "WAS": 14,
    "ORL": 15,

    # NORTHWEST
    "OKC": 16,
    "POR": 17,
    "UTA": 18,
    "DEN": 19,
    "MIN": 20,

    # PACIFIC
    "GSW": 21, 
    "LAC": 22,
    "SAC": 23,
    "PHO": 24,
    "LAL": 25,

    # SOUTH WEST
    "SAS": 26,
    "DAL": 27,
    "MEM": 28,
    "HOU": 29,
    "NOP": 30
}

# update those values
encoded_games_df = games_df

encoded_games_df.replace(team_encoding, inplace=True)

encoded_games_df



### Cumulative Average Logic    

In [ ]:
"""
Goal: We want to calculative the cumulative average of each column in the dataframe. We define the cumulative average as the
average of each particular stat (let's say points) up until (and not including) the game. 

Implications: This means that for the first game of each team of each season, each column will have a 0. 

Algorithm:
- We first need to identify the first date of each season we're parsing. This is because the cumulative average of each 
stat will be reset for each season

- For each season, we will leverage a hashmap to store each team's cumulative average. Quick Demo:

{
    Raptors: {
        num_games: 2
        cum_pts: 108
        cum_fgs: 12
        cum_rbs: 20
    }
}

- In this hashmap, we will be updating num_games everytime the particular team plays the game. 

"""



